# Pandas Draft Assistant

A code-based draft assistant (no UI).

In [1]:
import statisticaldrafting as sd

dm = sd.DraftModel(set="FDN")

In [9]:
# Draft assistant
collection = [] # Enter draft ids or cardnames
pick_order = dm.get_pick_order(collection)
pick_order

,name,rarity,color_identity,p1p1_rating,synergy,rating
162,"Liliana, Dreadhorde General",mythic,B,150.0,0.0,150.0
223,Sire of Seven Deaths,mythic,Colorless,141.5,0.0,141.5
35,Bloodthirsty Conqueror,mythic,B,140.7,0.0,140.7
91,Exemplar of Light,rare,W,138.9,0.0,138.9
52,Celestial Armor,rare,W,137.6,0.0,137.6
271,Vivien Reid,mythic,G,137.4,0.0,137.4
278,"Zimone, Paradox Sculptor",mythic,Multicolor,136.0,0.0,136.0
54,"Chandra, Flameshaper",mythic,R,135.3,0.0,135.3
138,High-Society Hunter,rare,B,134.5,0.0,134.5
63,Curator of Destinies,rare,U,134.2,0.0,134.2


In [ ]:
# This works. 
# from importlib.resources import files
# data_path = files('statisticaldrafting') / 'cards/cards.csv'